<a href="https://colab.research.google.com/github/jing20241015/NAFNET/blob/main/NAFNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 克隆NAFNet仓库
!git clone https://github.com/megvii-research/NAFNet.git
%cd NAFNet

Cloning into 'NAFNet'...
remote: Enumerating objects: 521, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 521 (delta 180), reused 142 (delta 142), pack-reused 296 (from 1)
Receiving objects: 100% (521/521), 16.19 MiB | 16.43 MiB/s, done.
Resolving deltas: 100% (277/277), done.
/content/NAFNet


In [13]:
# 确认一下PyTorch和CUDA是否已正确安装
!python -c "import torch; print('PyTorch版本:', torch.__version__); print('CUDA是否可用:', torch.cuda.is_available())"

# 安装BasicSR库
!pip install basicsr

# 安装其他可能的依赖
!pip install -r requirements.txt

PyTorch版本: 2.0.1+cu117
CUDA是否可用: True


In [11]:
!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.11/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        *****

In [4]:
import gdown
gdown.download('https://drive.google.com/uc?id=1Fr2QadtDCEXg6iwWX8OzeZLbHOx2t5Bj', "./experiments/pretrained_models/", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1Fr2QadtDCEXg6iwWX8OzeZLbHOx2t5Bj
From (redirected): https://drive.google.com/uc?id=1Fr2QadtDCEXg6iwWX8OzeZLbHOx2t5Bj&confirm=t&uuid=af8617e4-ae10-4bc6-ade6-3a7a44df1c73
To: /content/NAFNet/experiments/pretrained_models/NAFNet-GoPro-width32.pth
100%|██████████| 68.7M/68.7M [00:01<00:00, 39.8MB/s]


'./experiments/pretrained_models/NAFNet-GoPro-width32.pth'

In [6]:
import gdown
gdown.download('https://drive.google.com/uc?id=1zgALzrLCC_tcXKu_iHQTHukKUVT1aodI', "./datasets/GoPro/train", quiet=False)

gdown.download('https://drive.google.com/uc?id=1abXSfeRGrzj2mQ2n2vIBHtObU6vXvr7C', "./datasets/GoPro/test", quiet=False)

# 解压数据集
!mkdir -p datasets/GoPro/train datasets/GoPro/test
!unzip -q /content/NAFNet/datasets/GoPro/train/trainnjkgp3e_.part -d datasets/GoPro/train/
!unzip -q /content/NAFNet/datasets/GoPro/test/test4u9ch5rz.part -d datasets/GoPro/test/


Downloading...
From (original): https://drive.google.com/uc?id=1zgALzrLCC_tcXKu_iHQTHukKUVT1aodI
From (redirected): https://drive.google.com/uc?id=1zgALzrLCC_tcXKu_iHQTHukKUVT1aodI&confirm=t&uuid=5a11e375-10d6-4b05-b74a-9c851cd432b0
To: /content/NAFNet/datasets/GoPro/train
100%|██████████| 4.11G/4.11G [03:54<00:00, 17.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1abXSfeRGrzj2mQ2n2vIBHtObU6vXvr7C
From (redirected): https://drive.google.com/uc?id=1abXSfeRGrzj2mQ2n2vIBHtObU6vXvr7C&confirm=t&uuid=4f56fb42-d11c-4e4b-910d-1fe618274ba6
To: /content/NAFNet/datasets/GoPro/test
100%|██████████| 2.61G/2.61G [00:42<00:00, 61.0MB/s]


unzip:  cannot find or open /content/NAFNet/datasets/GoPro/train/trainnjkgp3e_.part, /content/NAFNet/datasets/GoPro/train/trainnjkgp3e_.part.zip or /content/NAFNet/datasets/GoPro/train/trainnjkgp3e_.part.ZIP.
unzip:  cannot find or open /content/NAFNet/datasets/GoPro/test/test4u9ch5rz.part, /content/NAFNet/datasets/GoPro/test/test4u9ch5rz.part.zip or /content/NAFNet/datasets/GoPro/test/test4u9ch5rz.part.ZIP.


In [15]:
!pip uninstall numpy -y
!pip install numpy==1.26.0

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 87.0 MB/s eta 0:00:00


In [8]:
# 卸载当前版本
!pip uninstall -y torch torchvision

# 安装兼容版本（使用适合CUDA 11.3的PyTorch版本）
!pip install torch==2.0.1 torchvision==0.15.2 --extra-index-url https://download.pytorch.org/whl/cu113

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
   ━━━━━

In [29]:
# 在 GoPro 数据集上测试 NAFNet-GoPro-width32
# 切换到NAFNet目录
#%cd /content/NAFNet

!python basicsr/test.py -opt ./options/test/GoPro/NAFNet-width32.yml

Disable distributed.
Path already exists. Rename it to /content/NAFNet/results/NAFNet-GoPro-width32-test_archived_20250408_134636
2025-04-08 13:46:36,385 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.2.0+2b4af71
	PyTorch: 2.0.1+cu117
	TorchVision: 0.15.2+cu117
2025-04-08 13:46:36,386 INFO: 
  name: NAFNet-GoPro-width32-test
  model_type: ImageRestorationModel
  scale: 1
  dist: False
  num_gpu: 1


In [ ]:
# 创建数据集目录
!mkdir -p datasets/DIV2K

# 下载DIV2K数据集
!wget https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!wget https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
!wget https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip
!wget https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X2.zip

# 解压数据集
!mkdir -p datasets/DIV2K/train datasets/DIV2K/val
!unzip -q DIV2K_train_HR.zip -d datasets/DIV2K/train/
!unzip -q DIV2K_train_LR_bicubic_X2.zip -d datasets/DIV2K/train/
!unzip -q DIV2K_valid_HR.zip -d datasets/DIV2K/val/
!unzip -q DIV2K_valid_LR_bicubic_X2.zip -d datasets/DIV2K/val/

--2025-04-07 12:30:04--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3530603713 (3.3G) [application/zip]
Saving to: ‘DIV2K_train_HR.zip’

DIV2K_train_HR.zip  100%[===================>]   3.29G  20.9MB/s    in 2m 47s  

2025-04-07 12:32:51 (20.2 MB/s) - ‘DIV2K_train_HR.zip’ saved [3530603713/3530603713]

--2025-04-07 12:32:51--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 925390592 (883M) [application/zip]
Saving to: ‘DIV2K_train_LR_bicubic_X2.zip’



In [30]:
#保存到Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [31]:
!cp -r /content/NAFNet /content/drive/MyDrive/NAFNET

In [32]:
!ls /content/drive/MyDrive/NAFNET

NAFNet


In [35]:
!git config --global user.email "1255743988@qq.com"
!git config --global user.name "jing20241015"
!git clone https://github.com/jing20241015/NAFNET

Cloning into 'NAFNET'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [38]:
# 1. 配置 SSH 密钥（如果未配置）
# 在 Colab 中生成 SSH 密钥并添加到 GitHub：
!ssh-keygen -t ed25519 -C "1255743988@qq.com"
!cat ~/.ssh/id_ed25519.pub  # 将输出的密钥复制到 GitHub 账户的 SSH 密钥设置中

# 2. 修改远程仓库地址为 SSH 格式
#!git remote set-url origin git@github.com:jing20241015/NAFNET.git

# 3. 测试 SSH 连接（确保密钥已正确配置）
#!ssh -T git@github.com

Generating public/private ed25519 key pair.
Enter file in which to save the key (/root/.ssh/id_ed25519): ^C
cat: /root/.ssh/id_ed25519.pub: No such file or directory


In [37]:
# 1. 进入项目根目录
%cd /content/NAFNet

# 2. 确认分支名称
#!git branch

# 3. 配置远程仓库（假设分支是 main）
!git remote set-url origin git@github.com:jing20241015/NAFNET.git  # 使用 SSH
# 或
#!git remote set-url origin https://github.com/jing20241015/NAFNET.git  # 使用 HTTPS

# 4. 提交更改（如果未提交）
!git add .
!git commit -m "Final commit"

# 5. 推送到远程仓库（根据分支名称选择）
!git push -u origin main  # 如果分支是 main
# 或
#!git push -u origin master  # 如果分支是 master

/content/NAFNet
Cloning into '/content/NAFNet/basicsr/models/NAFNET'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
[main 45dc87a] Convert NAFNET to submodule
 2 files changed, 4 insertions(+)
 create mode 100644 .gitmodules
 create mode 160000 basicsr/models/NAFNET
On branch main
nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/jing20241015/NAFNET'
